In [ ]:
from bs4 import BeautifulSoup
import json
import os
import requests

# Парсинг

In [ ]:
#@title Сбор всех страниц с темами форума
forums_page_list = []
response = requests.get('https://www.woman.ru/forum/?sort=30d')
count = 2
while response.status_code != 404:
    forums_page_list.append(response.text)
    response = requests.get(f'https://www.woman.ru/forum/{count}/?sort=30d')
    count += 1

In [ ]:
#@title Сбор ссылок на темы
href_list = []
for page in forums_page_list:
    soup = BeautifulSoup(page)
    links = soup.find_all(class_='list-item__link')
    for link in links:
        href_list.append(link['href'])


In [ ]:
#@title Парсинг самих сообщений
comments_list = []
for link in href_list:  # Идем по всем темам
    params = {}
    r = requests.post(f"https://www.woman.ru{href_list[0]}", params)
    temp_page_comments_list = []  # Список с блоками комментариев
    response = requests.get(f'https://www.woman.ru{link}', cookies=r.cookies)
    count = 2
    while response.status_code != 404:
        temp_page_comments_list.append(response.text)
        response = requests.get(f'https://www.woman.ru{link}{count}', cookies=r.cookies)
        count += 1
    for temp_page in temp_page_comments_list: # Идем по всем комментам и забираем только парные сообщение - ответ
        soup = BeautifulSoup(temp_page)
        comments_block = soup.find_all(class_='card__text')
        for comment in comments_block:
            if comment.find_all(class_='quote'):
                comments_list.append((comment.find_all(class_='quote__text')[0].text, comment.find_all(class_='card__comment')[0].text))
    # with open('cache.json', 'w') as f:
    #     json.dump(comments_list, f)            

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('cache.json', 'w', encoding='utf-8') as f:
    json.dump(comments_list, f, indent=4, ensure_ascii=False)

In [ ]:
with open('dataset.json', 'r', encoding='utf-8') as f:
    forum_data = json.load(f)
    

### Получившиеся данные

In [ ]:
from pandas.io.json import json_normalize

In [ ]:
formated_data = []
for item in forum_data:
    formated_data.append({'comment': item[0], 'answer': item[1]})

In [ ]:
with open('cache_formated.json', 'w', encoding='utf-8') as f:
    json.dump(formated_data, f, indent=4, ensure_ascii=False)

In [ ]:
with open('cache_formated.json', 'r', encoding='utf-8') as f:
    text = f.read()
    text = json.loads(text)

In [ ]:
json_normalize(text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,comment,answer
0,емуИ жвачка ещё 😁,"Агам))это я т создала,не для троллей,а чтоб ту..."
1,"Агам))это я тему создала,не для троллей,а чтоб...",Привет. Я поэтому и пришла)) 😁 я ж нормальная ...
2,Привет. Я поэтому и пришла)) 😁 я ж нормальная ...,"Прям как Джигурда в одном фильме кричал - ""Я н..."
3,"Прям как Джигурда в одном фильме кричал - ""Я н...",Типа того 😁😁😁😁 блин я аж интонацию его голоса ...
4,Типа того 😁😁😁😁 блин я аж интонацию его голоса ...,😂😂😂😂и я))
...,...,...
126102,1. Не то чтобы разбила. Скорее я был шокирован...,3. Да боже ж ты мой!!! На ком не женятся??? Ве...
126103,1. ))) Никто ничего не строит. Вы либо подходи...,"1. Ве, как же ты уже достал! Стоило жениться н..."
126104,1. Ну я же не взял её честь в 16 лет как она х...,"1. Нет, ты отнюдь не разумнее её. Хотя бы пото..."
126105,Я приличное количество человек встречала с так...,В уральской глубинке Ве живёт. Деревня Мурзинк...


# Нейронка

In [ ]:
! pip install datasets transformers[sentencepiece]
! pip install accelerate

In [ ]:
import sys
import re
import json

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_cosine_schedule_with_warmup

In [ ]:
#@title Чекпоинт предобученной модели с hugging face
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "Kirili4ik/ruDialoGpt3-medium-finetuned-telegram"   
tokenizer =  AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [ ]:
#@title Готовые функции для работы


def get_length_param(text: str, tokenizer) -> str:
    """Maps text to 1 of 4 buckets based on length after encoding.

    Parameters
    ----------
    text: str
        The text to be given 1 of 4 length parameters.

    tokenizer: HuggingFace tokenizer 
        Tokenizer that used to compute the length of the text after encoding.
        For more info ee https://huggingface.co/transformers/main_classes/tokenizer.html

    Returns
    -------
    len_param: str
        One of four buckets: 
        '1' for short, '2' for medium, '3' for long texts and '-' for all others. 
    """
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param


def get_user_param(text: dict, machine_name_in_chat: str) -> str:
    """Maps text by 1/0 for it to be the person or the machine in the dialogue

    Parameters
    ----------
    text: Dict[..., 'from', ...]
        Dict containing field 'from' with the name of the user who sent the message

    machine_name_in_chat: str
        Str with the name of the machine - it will be predicted
    """
    if text['from'] == machine_name_in_chat:
        return '1'  # machine
    else:
        return '0'  # human

# Здесь указываем ключ первого сообщения в датасете
def build_text_file(data_json: dict, dest_path: str, 
                    tokenizer, machine_name_in_chat='first'):
    """Create a text file for training in special format for ruDialoGPT-3.

    Parameters
    ----------
    data_json: dict
        Dict containing 'text' (message) and 'from' (user who sent the message)
        
    dest_path: str
        String containing path to write data there

    tokenizer: HuggingFace tokenizer 
        Tokenizer that used to compute the length of the text after encoding.
        For more info ee https://huggingface.co/transformers/main_classes/tokenizer.html
    """
    f = open(dest_path, 'w', encoding='utf-8')
    new_data = ''
    for i in range(len(data_json) - 1):
        message, next_message = data_json[i], data_json[i+1]
        if message['text'] == '' or type(message['text']) != str:
            continue
        if next_message['text'] == '' or type(next_message['text']) != str:
            continue

        user   = get_user_param(message, machine_name_in_chat=machine_name_in_chat)
        length = get_length_param(data_json[i+1]['text'], tokenizer)
        message_text = re.sub(r"\n", ". ", message['text'])
        new_data += f"|{user}|{length}|{message_text}{tokenizer.eos_token}" + "\n"

    f.write(new_data.encode('utf-8').decode('utf-8'))


def load_dataset(train_path, test_path, tokenizer):
    """Creates train and test PyTorch datasets and collate_fn using HuggingFace.

    Parameters
    ----------
    train_path: str
        String containing path to train data
        
    test_path: str
        String containing path to test data

    tokenizer: HuggingFace tokenizer 
        Tokenizer that used to compute the length of the text after encoding.
        For more info ee https://huggingface.co/transformers/main_classes/tokenizer.html
    """
    train_dataset = TextDataset(
          tokenizer  = tokenizer,
          file_path  = train_path,
          block_size = 256)
     
    test_dataset = TextDataset(
          tokenizer  = tokenizer,
          file_path  = test_path,
          block_size = 256)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )
    return train_dataset, test_dataset, data_collator

In [ ]:
with open('dataset.json', 'r', encoding='utf-8') as f:
    forum_data = json.load(f)

In [ ]:
#@title Подгоняем данные в нужный формат
formated_data = []
for item in forum_data:
    formated_data.append({'from': 'first', 'text': item[0]})
    formated_data.append({'from': 'second', 'text': item[1]})

In [ ]:
#@title Сборка файлов
machine_name_in_chat = 'first' #@param {type : "string"}

data = formated_data

# test data is first 10% of chat, train - last 90%
train, test = data[int(len(data)*0.1):], data[:int(len(data)*0.1)]

build_text_file(train, 'train_dataset.txt', tokenizer)
build_text_file(test,  'test_dataset.txt', tokenizer)

print("Train dataset length: " + str(len(train)) + "samples")
print("Test dataset length: "  + str(len(test)) + "samples")

Train dataset length: 226993samples
Test dataset length: 25221samples


In [ ]:
# let's look at our data
! head -n 10 train_dataset.txt

|0|1|Милый на этих выходных очень отличился. Очень. И одновременно с этим множество вот этих мозговзрывных моментов. Я в замешательстве буквально. Не представляю что дальше будет.</s>
|1|2|Ну все,Ань,это ппипец полный тогда))</s>
|0|2|Соньчик, это вообще. В феврале год нашего знакомства. И только вчера я стала "постоянная тёла"))</s>
|1|2|Я так не думаю. Это не означает, что мои чувства должны выключиться. Господи, у него даже выражение лица, как у отца, ужас🤯😱</s>
|0|2|Не придавай этому значение, это нормально. Ну похож и похож.Главное, не думай, что это какая-то карма, а просто осознавай, что это заложено от природы. Всё в порядке 😊</s>
|1|2|Милый на этих выходных очень отличился. Очень. И одновременно с этим множество вот этих мозговзрывных моментов. Я в замешательстве буквально. Не представляю что дальше будет.</s>
|0|2|Может,ему просто нужно было выплеснуть?И всё таки он не такой плохой?)Не знаю Ань,решать тебе,а то мы тут насоветуем))</s>
|1|2|Соньчик, это вообще. В феврале год н

In [ ]:
# Create PyTorch Datasets
train_dataset, test_dataset, data_collator = load_dataset('train_dataset.txt', 'test_dataset.txt', tokenizer)

# Create PyTorch Dataloaders
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=3, collate_fn=data_collator)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
# this cell checks 1 forward pass
try:
    for batch in train_loader:
        break
    {k: v.shape for k, v in batch.items()}

    outputs = model(**batch)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

In [ ]:
#@title Работа сети
chat_history_ids = torch.zeros((1, 0), dtype=torch.int)

while True:
    
    next_who = input("Who's phrase?\t")  #input("H / G?")     # Human or GPT

    # In case Human
    if next_who == "H":
        input_user = input("===> Human: ")
        
        # encode the new user input, add parameters and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(f"|0|{get_length_param(input_user, tokenizer)}|" \
                                              + input_user + tokenizer.eos_token, return_tensors="pt")
        # append the new user input tokens to the chat history
        chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

    if next_who == "G":

        next_len = input("Phrase len? 1/2/3/-\t")  #input("Exp. len?(-/1/2/3): ")
        # encode the new user input, add parameters and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(f"|1|{next_len}|", return_tensors="pt")
        # append the new user input tokens to the chat history
        chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
        
        #print(tokenizer.decode(chat_history_ids[-1])) # uncomment to see full gpt input
        
        # save previous len
        input_len = chat_history_ids.shape[-1]
        # generated a response; PS you can read about the parameters at hf.co/blog/how-to-generate
        chat_history_ids = model.generate(
            chat_history_ids,
            num_return_sequences=1,                     # use for more variants, but have to print [i]
            max_length=512,
            no_repeat_ngram_size=3,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature = 0.6,                          # 0 for greedy
            mask_token_id=tokenizer.mask_token_id,
            eos_token_id=tokenizer.eos_token_id,
            unk_token_id=tokenizer.unk_token_id,
            pad_token_id=tokenizer.pad_token_id,
            device='cpu'
        )
        
        # pretty print last ouput tokens from bot
        print(f"===> GPT-3:  {tokenizer.decode(chat_history_ids[:, input_len:][0], skip_special_tokens=True)}")

Who's phrase?	H
===> Human: Привет
Who's phrase?	G
Phrase len? 1/2/3/-	1
===> GPT-3:  Ну вот я не знаю
Who's phrase?	H
===> Human: Какой ты любишь чай?
Who's phrase?	G
Phrase len? 1/2/3/-	3
===> GPT-3:  Мне нравится чёрный
Who's phrase?	H
===> Human: А еще?
Who's phrase?	G
Phrase len? 1/2/3/-	2
===> GPT-3:  Ну я не особо любитель, но типа вот у меня дома есть и черный и зеленый и тд


KeyboardInterrupt: ignored